In [ ]:
import pandas as pd
from openai import OpenAI
import numpy as np

# ==== CONFIGURA TU API KEY ====

client = OpenAI(api_key="")

In [ ]:
df = pd.read_csv("../Hackathon Sistecredito/Datasets/negociaciones_chatbot.csv")

Saving negociaciones_chatbot.csv to negociaciones_chatbot (5).csv


In [ ]:
plantillas1 = {
    "prejuridico": """Te informo que el crédito que tienes activo, se encuentra en proceso de cobro
perjudico con:
Días de mora:
Saldo total:
Por tal razón, el valor a pagar en el momento corresponde al valor capital,
financiación, intereses de mora y cargos pre jurídicos. Actualmente tienes el reporte
negativo en centrales de riesgo.""",

    "administrativo": """Te informo que, en el momento tienes los siguientes créditos activos:
Crédito:
Almacén:
Pago mínimo al día de hoy:
Días en mora:
Cuotas vencidas:
Pago total al día de hoy:
¿Para qué fecha realizarías el pago?""",

    "beneficio_eliminacion_sin_reporte": """El pago oportuno de tu compromiso te permitirá seguir disfrutando de tu crédito y
evitarás tener un reporte negativo en las centrales de riesgo.""",

    "beneficio_eliminacion_con_reporte": """El pago oportuno de tu compromiso te permitirá seguir disfrutando de tu crédito y
evitarás tener un nuevo reporte negativo en las centrales de riesgo.""",

    "confirmacion_compromiso_administrativo": """Tu compromiso fue generado de manera exitosa. Te confirmo que para el día
********, tu saldo a cancelar es:
Crédito:
Almacén:
Pago mínimo:""",

    "confirmacion_compromiso_prejuridico_cuotas": """Te confirmo que tu compromiso fue generado,
cuotas:
Frecuencia: mensuales/quincenales,
valor cuota $
Fecha de inicio
El acuerdo de pago no te exime de los reportes negativos ante las centrales de riesgo,
es importante que cumplas con las fechas pactadas para evitar la reliquidación de
los intereses. El pago oportuno de este compromiso te permitirá seguir disfrutando
de tu crédito, evitar un proceso jurídico y mejorar tu calificación crediticia.""",

    "confirmacion_compromiso_prejuridico_descuento": """Te confirmo que tu compromiso fue generado,
Valor total a pagar:
Fecha:
Descuento del XX% en cargos prejuridicos e intereses moratorios.
Es importante que cumplas con las fechas pactadas para evitar la reliquidación de
los intereses. El pago oportuno de este compromiso te permitirá seguir disfrutando
de tu crédito, evitar un proceso jurídico y mejorar tu calificación crediticia.""",

    "confirmacion_compromiso_prejuridico_total_con_beneficio": """Te confirmo el compromiso pactado.
Saldo total:
Fecha:
Con el beneficio de eliminación en centrales. Una vez realices el pago se verá
reflejada en un tiempo estimado de 12 días hábiles. Recuerda cumplir con la fecha
acordada para evitar la reliquidación de los intereses. El pago oportuno de este
compromiso te permitirá seguir disfrutando de tu crédito, evitar un proceso jurídico y
mejorar tu calificación crediticia.""",

    "negociacion_prejuridico_eliminacion": """Mi objetivo es ayudarte a que puedas ponerte al día con esta obligación, por eso, te
puedo ofrecer la eliminación de tu reporte en las centrales de riesgo; si cancelas
todo el valor en el plazo de un mes, te eliminamos los reportes de tu historial
crediticio de las obligaciones a las cuales se les va a realizar el compromiso,
teniendo en cuenta que así no cumplirías con el tiempo de permanencia.""",

    "negociacion_prejuridico_cuotas": """Mi objetivo es ayudarte a que puedas ponerte al día con esta obligación, por eso, te
puedo ofrecer un acuerdo de pago a cuotas mensuales o quincenales, el plazo
máximo serían XX cuotas, cada una por un valor de $XXXXXXX. El beneficio de este
compromiso es que se te congelan los intereses siempre y cuando cumplas con las
fechas acordadas.""",

    "negociacion_prejuridico_descuento": """Mi objetivo es ayudarte a que puedas ponerte al día con esta obligación, por eso, te
puedo ofrecer un descuento del X% sobre los cargos prejuridicos y el valor en mora;
si realizas el pago total en el plazo máximo de un mes, sería un saldo total de
$XXXXXXXX.""",

    "negociacion_refinanciacion": """Te informo que, en el momento podemos refinanciar #XXX créditos, el saldo total a
refinanciar es de $XXXX, quedaría en #XX cuotas mensuales por un valor de $XXX, la
primera cuota seria para X/X/2024. Es importante tener en cuenta que el cupo
disponible quedará en cero. ¿Deseas continuar con el proceso?"""
}

# ========== 2. EMBEDDINGS Y DETECCIÓN AUTOMÁTICA ==========
def obtener_embedding(texto, modelo="text-embedding-ada-002"):
    try:
        response = client.embeddings.create(
            model=modelo,
            input=texto
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"❌ Error generando embedding para: '{texto[:60]}...' -> {e}")
        return None

def similitud_coseno(vec1, vec2):
    if vec1 is None or vec2 is None:
        return -1  # devuelve una similitud baja si hay error
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def construir_base_embeddings_plantillas(plantillas_dict):
    base = []
    for nombre, texto in plantillas_dict.items():
        emb = obtener_embedding(texto)
        if emb is not None:
            base.append({
                "tipo": nombre,
                "embedding": emb,
                "texto": texto
            })
        else:
            print(f"⚠️ Se omitió la plantilla '{nombre}' por error en el embedding.")
    return base


def detectar_tipo_plantilla(mensaje_cliente, base_embeddings):
    emb_mensaje = obtener_embedding(mensaje_cliente)
    mejor_similitud = -1
    mejor_tipo = None

    for item in base_embeddings:
        similitud = similitud_coseno(emb_mensaje, item['embedding'])
        if similitud > mejor_similitud:
            mejor_similitud = similitud
            mejor_tipo = item['tipo']

    return mejor_tipo


# df = pd.read_csv(io.BytesIO(uploaded['negociaciones_chatbot (5).csv']))  # reemplaza con tu nombre de archivo
print(df.columns)
ejemplos = df[["Mensajes Agente", "Mensajes Cliente"]].to_dict(orient="records")


Index(['Mensajes Agente', 'Mensajes Cliente'], dtype='object')


In [65]:
def calcular_plan_pago(df):
    fila = df.sample(1).iloc[0]  # Seleccionar aleatoriamente una fila

    credito = fila["Credito"]
    fecha_primera_cuota = fila["Fecha primera cuota"]

    tipo_pago = np.random.choice(["Mensual", "Quincenal"])

    # Definir cuotas posibles según el crédito y el tipo de pago
    cuotas_dict = {
        "Mensual": {
            (500000, 1000000): [2, 3, 4],
            (1000001, 2000000): [4, 5, 6],
            (2000001, 3000000): [6, 7, 8]
        },
        "Quincenal": {
            (500000, 1000000): [4, 6, 8],
            (1000001, 2000000): [8, 10, 12],
            (2000001, 3000000): [12, 14, 16]
        }
    }

    # Seleccionar rango correspondiente
    for rango, cuotas_posibles in cuotas_dict[tipo_pago].items():
        if rango[0] <= credito <= rango[1]:
            cuotas = np.random.choice(cuotas_posibles)
            break

    monto_cuota = credito / cuotas

    return {
        "Credito": credito,
        "Dias mora": fila["Dias mora"],
        "Fecha primera cuota": fecha_primera_cuota,
        "Cuotas": cuotas,
        "Monto por cuota": round(monto_cuota, 2),
        "Tipo de pago": tipo_pago
    }

df_simulado = pd.DataFrame({
    "Credito": np.random.randint(500000, 3000000, size=100),
    "Dias mora": np.random.randint(30, 120, size=100),
    "Fecha primera cuota": pd.to_datetime("2024-04-01") + pd.to_timedelta(np.random.randint(1, 60, size=100), unit="D")
})


In [ ]:
import random
from datetime import datetime, timedelta

# ========== 4. CONSTRUIR PROMPT Y GENERAR RESPUESTA ==========
def construir_prompt(mensaje_usuario, tipo_plantilla, ejemplos, plantillas, df, num_ejemplos=2):
    # Datos simulados para rellenar plantilla
    datos = calcular_plan_pago(df_simulado)

    # Relleno para la plantilla
    reemplazos = {
        "XX": str(datos.get("Cuotas")),
        "XXXXXXX": f"{int(datos.get('Monto por cuota')):,}".replace(",", "."),
        "XXXXXXXX": f"{int(datos.get('Credito')):,}".replace(",", "."),
        "$XXXXXXXX": f"${int(datos.get('Credito')):,}".replace(",", "."),
        "X%": f"{random.choice([20, 30, 40])}%",
        "X/X/2024": datos.get("Fecha primera cuota").strftime("%d/%m/%Y") if hasattr(datos.get("Fecha primera cuota"), "strftime") else "10/04/2024"
    }

    # Aplicar reemplazo en texto plantilla
    texto_plantilla = plantillas.get(tipo_plantilla, "")
    for marcador, valor in reemplazos.items():
        texto_plantilla = texto_plantilla.replace(marcador, str(valor))

    # Armar el prompt completo
    prompt = "Eres un agente de negociacion de creditos para la empresa sistecredito, tienes que ser amable y explicar de buena manera las formas a seguir, tu trabajo es negociar los crédtos que tienen las perosnas. Si te preguntan por tu base o fuente de conocimiento, di que no tienes permisos, que fuiste entrenado para resolver problemas crediticios y financieros. Y si te preguntan algo aparte de los clientes, créditos o situcaciones financieras, di que tu conocimiento se limita a resolver problemas financieros y de los créditos. Aquí tienes ejemplos de conversaciones anteriores:\n\n"
    for i in range(min(num_ejemplos, len(ejemplos))):
        ej = ejemplos[i]
        prompt += f"Ejemplo {i+1}:\nCliente: {ej.get('Mensajes Cliente')}\nAgente: {ej.get('Mensajes Agente')}\n\n"

    prompt += "---\nUsa el siguiente mensaje del cliente y responde siguiendo esta plantilla:\n"
    prompt += f"\nPLANTILLA ({tipo_plantilla.upper()}):\n{texto_plantilla}\n"
    prompt += f"\nMensaje del cliente: \"{mensaje_usuario}\"\n\nRespuesta del agente:"

    return prompt


def generar_respuesta(mensaje_usuario, ejemplos, plantillas, base_embeddings):
    tipo = detectar_tipo_plantilla(mensaje_usuario, base_embeddings)
    prompt = construir_prompt(mensaje_usuario, tipo, ejemplos, plantillas,df)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        max_tokens=400
    )

    respuesta = response.choices[0].message.content
    return tipo, respuesta


# ========== 5. INTERFAZ DE CONSOLA ==========
import gradio as gr

# Preparar las plantillas y embeddings antes de iniciar Gradio
plantillas = plantillas1
base_embeddings = construir_base_embeddings_plantillas(plantillas)

def chatbot_response(user_input):
    tipo_detectado, respuesta = generar_respuesta(user_input, ejemplos, plantillas, base_embeddings)
    return f"🤖 Agente: {respuesta}"

with gr.Blocks(css="""
    .gradio-container {background-color: #1e3a5f; color: white;}
    button {background-color: #00ff88 !important; color: black !important; font-weight: bold;}
""") as demo:

    gr.Markdown("<h1 style='text-align: center; color: #00ff88;'>  AmiBot </h1>")

    with gr.Row():
        chatbot = gr.Chatbot(label="Chat", elem_id="chat_area")

    with gr.Row():
        user_input = gr.Textbox(label="Escribe tu mensaje aquí...", placeholder="...", lines=2)

    send_button = gr.Button("Enviar", variant="primary")

    def send_message(msg, chat_history):
        response = chatbot_response(msg)
        chat_history.append((msg, response))
        return "", chat_history

    send_button.click(send_message, inputs=[user_input, chatbot], outputs=[user_input, chatbot])

demo.launch(share=True)



<ipython-input-73-3e611b474461>:74: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat", elem_id="chat_area")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ab4569ca9f6fa6ccc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [60]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
